In [4]:
pip install tika

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pdfminer.six

Note: you may need to restart the kernel to use updated packages.


In [1]:
from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

In [2]:
import re
import codecs
import pandas as pd
import os, glob
from tika import parser 
from pandas import DataFrame
import PyPDF2

# A class is created to extract texts from PDF files

class PdfExtracter:
    
    
    def __init__(self, 
                PATH:str="C:\\Users\\Sangamithra\\Desktop\\PDFs",
                files:list=[],
                lis:list=[]
               ):
        
       
        self.PATH = PATH
        self.files=files
        self.lis=lis

        #File extraction method extracts files from the PDF document and prints the document information 
        
    def fileextraction(self):
        for file in glob.glob(self.PATH+ "/*.pdf"):
            if file.endswith('.pdf'):
                self.files.append(file)
        for i in range(len(self.files)):
            print('PDF DOCUMENT NAME: '+ str(self.files[i]))
            file_open = open(self.files[i], 'rb')
            fileReader = PyPDF2.PdfFileReader(file_open)
            print('Number of pages: ' + str(fileReader.numPages))
            print('Documentation Info: '+ str(fileReader.documentInfo))
            
            # Pdf is converted to string and appended to a list
            
            def convert_pdf_to_string(file_path):
                output_string = StringIO()
                with open(file_path, 'rb') as in_file:
                    parser = PDFParser(in_file)
                    doc = PDFDocument(parser)
                    rsrcmgr = PDFResourceManager()
                    device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
                    interpreter = PDFPageInterpreter(rsrcmgr, device)
                    for page in PDFPage.create_pages(doc):
                        interpreter.process_page(page)
                return(output_string.getvalue())
            text=convert_pdf_to_string(self.files[i])
            print(" -------------------------------------------------------------------------------")
            self.lis.append(text)
  #The list is passed  to data cleaning method and removes special characters from the string, Removes white spaces and stores in mod_string
    def datacleaning(self):
        content=[]
        for i in range(len(self.lis)):
            content.append(self.lis[i].replace('\n',''))
        mod_string=''.join([str(item) for item in content])
        list_of_char = ["⇑","þ","ð","Þ","ﬃ","ø","•","@",":","+","–","â","€","™","#","}","~","=",";","(","(cid","cid",")","⁄","^","¼","1","2","3","4","5","6","7","8","9","0","/","com","Fig.","%" ,"[","]","?",">","<","±","$","-","©"]
        for character in list_of_char:
            mod_string = a=mod_string.replace(character, "")
        mod_string=re.sub("\s\s+",' ',mod_string)
        mod_string=mod_string.replace("i.e.", "That is,") 
       # print(mod_string)
        final=self.textfile(mod_string)
    
    #the modified string is then passed to the textfile and converted to csv
            
    def textfile(self,mod_string):
            with codecs.open("pdf_extracted.txt", "w", "utf-8-sig") as temp:
                temp.write(mod_string)
            df = pd.read_csv("pdf_extracted.txt",encoding = 'utf-8-sig',header=None,sep='.')
            
            df=df.T
            df.to_csv('pdf_extracted.csv')
            csv_row=self.rowreduction()
    
    # Data cleaning at csv file
            
    def rowreduction(self):
        df = pd.read_csv('pdf_extracted.csv')
        new_df = df[df['0'].str.len() <= 20]
        cond = df['0'].isin(new_df['0'])
        df.drop(df[cond].index, inplace = True)
        df.to_csv('pdf_extracted.csv')
        csv_drop=self.emptyspace()
   
    def emptyspace(self):
        df = pd.read_csv('pdf_extracted.csv')
        df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
      
        df.dropna().to_csv('pdf_extracted.csv', index=False)
        

In [3]:

def main():
    pdf_obj=PdfExtracter()
    pdf_obj.fileextraction()
    pdf_obj.datacleaning()
    
   
if __name__=="__main__":
    main()


PDF DOCUMENT NAME: C:\Users\Sangamithra\Desktop\PDFs\07866141-annotated.pdf
Number of pages: 5
Documentation Info: {'/CreationDate': "D:20160519161543-07'00'", '/IEEE#20Article#20ID': '7866141', '/IEEE#20Issue#20ID': '7866072', '/IEEE#20Publication#20ID': '7864767', '/Meeting#20Ending#20Date': '16 June 2016', '/Meeting#20Starting#20Date': '13 June 2016', '/ModDate': "D:20200617102509-04'00'", '/Producer': 'iTextSharp 4.0.7 (based on iText 2.0.7); modified using iText® 5.5.6 ©2000-2015 iText Group NV (AGPL-version)', '/Subject': '2016 IEEE First International Conference on Data Science in Cyberspace (DSC);2016; ; ;10.1109/DSC.2016.15', '/Title': 'OPSDS: A Semantic Data Integration and Service System Based on Domain Ontology'}
 -------------------------------------------------------------------------------
PDF DOCUMENT NAME: C:\Users\Sangamithra\Desktop\PDFs\1-s2.0-S0020025512002587-main-annotated.pdf
Number of pages: 24
Documentation Info: {'/Author': 'Mustansar Ali Ghazanfar', '/Creati